In [11]:
#####################################################################################################################
## LDA-COHERENCE-ZILLA  
## Fuentes : https://markroxor.github.io/gensim/static/notebooks/topic_coherence_tutorial.html
##             https://medium.com/@lettier/how-does-lda-work-ill-explain-using-emoji-108abf40fa7d
##             http://phusewiki.org/wiki/images/c/c9/Weizhong_Presentation_CDER_Nov_9th.pdf
##             The Art and Science of Analyzing Sotfware Data, Christian Bird et al
##             https://rstudio-pubs-static.s3.amazonaws.com/79360_850b2a69980c4488b1db95987a24867a.html
##             https://www.quora.com/What-should-I-clean-from-text-before-LDA
##             https://www.machinelearningplus.com/nlp/lemmatization-examples-python/
##             https://github.com/derekgreene/topic-model-tutorial/blob/master/3%20-%20Parameter%20Selection%20for%20NMF.ipynb
##             https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/
##             https://radimrehurek.com/gensim/models/ldamodel.html
##             https://radimrehurek.com/gensim/models/coherencemodel.html
##             https://towardsdatascience.com/light-on-math-machine-learning-intuitive-guide-to-latent-dirichlet-allocation-437c81220158
##             https://towardsdatascience.com/evaluate-topic-model-in-python-latent-dirichlet-allocation-lda-7d57484bb5d0
##             https://towardsdatascience.com/latent-dirichlet-allocation-lda-9d1cd064ffa2
##             https://stats.stackexchange.com/questions/59684/what-are-typical-values-to-use-for-alpha-and-beta-in-latent-dirichlet-allocation
##             https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0
##             https://medium.com/@yanlinc/how-to-build-a-lda-topic-model-using-from-text-601cdcbfd3a6
#############################################by#######JMG############2020################################################
## Estructura del Código:
## - Importación de librerias
## - Funciones
## - Carga de archivo CSV
## - Limpieza
## - Segmentación de noticias por semana desde el 13-03 al 14-04
## - Visualización - Semana 1 
## - Visualización - Semana 2
## - Visualización - Semana 3
## - Visualización - Semana 4
## - Visualización - ALL


import pandas as pd
import numpy as np
import re
import gensim.corpora as corpora
import gensim
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess
import spacy
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
import pyLDAvis.gensim
import pickle 
import pyLDAvis
from pprint import pprint




In [3]:
## - Funciones

def remove_stopwords(texts):
    nltk.download('stopwords')
    stop_words = stopwords.words('spanish')
    newStopWords = ['coronavirus','chile','casos','detalles','agrego','embargo','forma','semanas','fecha','francia','espana','cada','agregó','parte','caso','dias','respecto','momento','covid','ascom','tikitakas','personas','tipo','tenía','jorge','habrá','habra','medios','perú','casa','cuales',
                'marzo','abril','lunes','semana','despues','hora','también','tambien','martes','radio','lugar','lugares','anuncio','anunciar',
                'miércoles','jueves','viernes','sábado','domingo','enero','comentarios','reservamos','febrero','italia','según','segun','horas','aquí','virus','veces','radio','eliodoro', 'yanez', 'providencia',
                 'reservamos','derecho','fabiola','shtml','pictwittercom','arrow','biobiochile','https_www','bloquear','april','usuarios','agresivos','commons','noticias','indique','contrario','eliminar','comentarios','debate','espacio','wordpress','desarrollado','detallo','estime','conveniente','manteniendo',]
    stop_words.extend(newStopWords)    
    
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def make_bigrams(texts,bigram_mod):
    return [bigram_mod[doc] for doc in texts]


def to_array(texts):
    arreglo_v=[]
    for docv in range(len(texts)):
        for wordv in data_lemmatized[docv].split():
            if wordv is None:
                pass
            else:
                arreglo_v.append(wordv)
    return arreglo_v
def lemma_sustantivos(texts):
    nlp1 = spacy.load("es_core_news_sm")
    text_out = []
    for sent in texts:
        doc = nlp1(" ".join(sent)) 
        text_out.append(" ".join([token.text for token in doc.noun_chunks])) 
    return text_out

def coherence_zilla(corpus,dictionary,dataset,texts,tmin,tmax):
    
    min_topics = tmin
    max_topics = tmax
    step_size = 1
    topics_range = range(min_topics, max_topics, step_size)
    alpha = list(np.arange(0.01, 1, 0.3))
    beta  = list(np.arange(0.01,0.1, 0.03))
    num_of_docs = len(corpus)
    max_coherence = 0
    best_model = dict();

    corpus_sets = gensim.utils.ClippedCorpus(corpus, num_of_docs*0.8),
    for k in topics_range:
        for a in alpha:
            for b in beta:
                lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=50,
                                           passes=10,
                                           alpha=a,
                                           eta=b,
                                           per_word_topics=True)
                              
                coherence_model_lda = CoherenceModel(model=lda_model, corpus=corpus,texts=texts, coherence='c_v')
                cv = coherence_model_lda.get_coherence()
                if max_coherence < cv:
                    max_coherence=cv
                    best_alpha=a
                    best_beta=b
                    best_topics=k
                  
                else:
                    pass
         
                    
      
    best_model['coherence'] = max_coherence
    best_model['alpha']   = best_alpha
    best_model['beta'] = best_beta
    best_model['topics'] =  best_topics
    
    return best_model

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
        
def pre_proceso(df_corpus):
    datos= df_corpus.values.tolist()
    datos_palabras= list(sent_to_words(datos))
    
    bigram = gensim.models.Phrases(datos_palabras,min_count=5) 
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    data_words_nostops = remove_stopwords(datos_palabras)
    data_words_bigrams = make_bigrams(data_words_nostops,bigram_mod)
    data_lemmatized =  lemma_sustantivos(data_words_bigrams)
    return data_lemmatized
            

In [4]:
## - Carga de archivo CSV

df_file = pd.read_csv('cleaning_contenido3.csv')

In [5]:
## - Limpieza

df_corpus= df_file
df_corpus=df_corpus[1:]
df_corpus=df_corpus.dropna()
df_corpus['contenido'] = df_corpus['contenido'].map(lambda x: re.sub('\s+', ' ', x))
df_corpus['contenido'] = df_corpus['contenido'].map(lambda x: re.sub('[_,";¿*&()=:+#\|@.!?]', ' ', x))
df_corpus['contenido'] = df_corpus['contenido'].map(lambda x: x.lower())
palabras_cortas = re.compile(r'\W*\b\w{1,3}\b')
df_corpus['contenido'] = df_corpus['contenido'].map(lambda x: palabras_cortas.sub('', x))

df_corpus['titulos'] = df_corpus['titulos'].map(lambda x: re.sub('\s+', ' ', x))
df_corpus['titulos'] = df_corpus['titulos'].map(lambda x: re.sub('[_,";¿*&()=:+#\|@.!?]', ' ', x))
df_corpus['titulos'] = df_corpus['titulos'].map(lambda x: x.lower())
palabras_cortas = re.compile(r'\W*\b\w{1,3}\b')
df_corpus['titulos'] = df_corpus['titulos'].map(lambda x: palabras_cortas.sub('', x))
df_corpus['contenido'] = df_corpus['titulos']+" "+df_corpus['contenido']




<>:6: DeprecationWarning: invalid escape sequence \s
<>:7: DeprecationWarning: invalid escape sequence \|
<>:12: DeprecationWarning: invalid escape sequence \s
<>:13: DeprecationWarning: invalid escape sequence \|
<>:6: DeprecationWarning: invalid escape sequence \s
<>:7: DeprecationWarning: invalid escape sequence \|
<>:12: DeprecationWarning: invalid escape sequence \s
<>:13: DeprecationWarning: invalid escape sequence \|
<>:6: DeprecationWarning: invalid escape sequence \s
<>:7: DeprecationWarning: invalid escape sequence \|
<>:12: DeprecationWarning: invalid escape sequence \s
<>:13: DeprecationWarning: invalid escape sequence \|
<ipython-input-5-99e369207d15>:6: DeprecationWarning: invalid escape sequence \s
  df_corpus['contenido'] = df_corpus['contenido'].map(lambda x: re.sub('\s+', ' ', x))
<ipython-input-5-99e369207d15>:7: DeprecationWarning: invalid escape sequence \|
  df_corpus['contenido'] = df_corpus['contenido'].map(lambda x: re.sub('[_,";¿*&()=:+#\|@.!?]', ' ', x))
<ipy

In [6]:
## - Segmentación de noticias por semana desde el 13-03 al 14-04

df_semana_1 = df_corpus[(df_corpus['fecha publicacion'] >= '2020-03-01') & (df_corpus['fecha publicacion'] <= '2020-03-23')]
df_semana_2 = df_corpus[(df_corpus['fecha publicacion'] > '2020-03-23') & (df_corpus['fecha publicacion'] <= '2020-03-30')]
df_semana_3 = df_corpus[(df_corpus['fecha publicacion'] > '2020-03-30') & (df_corpus['fecha publicacion'] <= '2020-04-07')]
df_semana_4 = df_corpus[(df_corpus['fecha publicacion'] > '2020-04-07') & (df_corpus['fecha publicacion'] <= '2020-04-14')]


In [8]:
############# Visualización - Semana 1 ##########
s_lemma_1 = pre_proceso(df_semana_1)
dataset_1 = [d.split() for d in s_lemma_1]
id2word_1 = corpora.Dictionary(dataset_1)
texts_1 = dataset_1
corpus_1 = [id2word_1.doc2bow(text) for text in texts_1]

s1_model = coherence_zilla(corpus_1,id2word_1,dataset_1,texts_1,10,16)
lda_model_s1 = gensim.models.LdaMulticore(corpus=corpus_1,
                                       id2word=id2word_1,
                                       num_topics=s1_model.get('topics'), 
                                       alpha=s1_model.get('alpha'), 
                                       eta=s1_model.get('beta'), 
                                       random_state=100,
                                       chunksize=50,
                                       passes=10,
                                       per_word_topics=True)
    
pyLDAvis.enable_notebook()
LDAvis_prepared_1 = pyLDAvis.gensim.prepare(lda_model_s1, corpus_1,id2word_1)
LDAvis_prepared_1



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jmerc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\ProgramData\Anaconda3\envs\simple\lib\site-packages\msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)
C:\ProgramData\Anaconda3\envs\simple\lib\site-packages\msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)
C:\ProgramData\Anaconda3\envs\simple\lib\site-packages\msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)
C:\ProgramData\Anaconda3\envs\simple\lib\site-packages\msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)
C:\ProgramData\Anaconda3\envs\simple\lib\site-packages\msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated,

15
0.3380993555224245
0.34120437532581993
0.3638232271018202
0.3659358249611964


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.017771  0.110427       1        1  47.578548
10     0.217146  0.002969       2        1  35.754101
7     -0.030143  0.076289       3        1   3.966825
6      0.001523 -0.073340       4        1   2.971935
5     -0.039833 -0.058497       5        1   2.887390
9     -0.033556 -0.029743       6        1   2.055752
3     -0.022669 -0.004232       7        1   1.871968
12    -0.000222  0.000479       8        1   0.785840
8     -0.022240 -0.006861       9        1   0.758673
4     -0.008302 -0.002537      10        1   0.228163
13    -0.008455 -0.002881      11        1   0.228163
14    -0.009103 -0.002463      12        1   0.228162
11    -0.008588 -0.003055      13        1   0.228162
2     -0.009105 -0.003126      14        1   0.228162
0     -0.008685 -0.003429      15        1   0.228162, topic_info=           Term       Freq      Total Category  logprob  loglift
38        salud  47.000000  47.000000  Default  30.0000  30.0000
11      medidas  48.000000  48.000000  Default  29.0000  29.0000
18      persona  10.000000  10.000000  Default  28.0000  28.0000
19       region  15.000000  15.000000  Default  27.0000  27.0000
9       informo  13.000000  13.000000  Default  26.0000  26.0000
..          ...        ...        ...      ...      ...      ...
374    pandemia   0.006227  22.276731  Topic15  -7.0934  -2.0995
212  enfermedad   0.006195  15.811373  Topic15  -7.0986  -1.7619
397        plan   0.006191  15.917475  Topic15  -7.0991  -1.7691
358        nota   0.006188  11.369489  Topic15  -7.0996  -1.4331
149     comunas   0.006182  11.372568  Topic15  -7.1007  -1.4345

[916 rows x 6 columns], token_table=      Topic      Freq             Term
term                                  
55        1  0.882290         acciones
709       2  0.941509         acuerdos
533       1  0.561985     adolescentes
60        1  0.551876  adultos_mayores
60        2  0.344923  adultos_mayores
...     ...       ...              ...
524       1  0.814336         vigencia
524       5  0.203584         vigencia
1010      2  0.998599  wwwipsenlineacl
1243      3  1.037837          yoihiay
1011      2  1.013701             zona

[428 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 11, 8, 7, 6, 10, 4, 13, 9, 5, 14, 15, 12, 3, 1])

In [13]:
# 10 topicos -semana 1
pprint(lda_model_s1.print_topics())
print(s1_model)

NameError: name 'lda_model_s1' is not defined

In [8]:
############# Visualización - Semana 2 ##########
s_lemma_2 = pre_proceso(df_semana_2)
dataset_2 = [d.split() for d in s_lemma_2]
id2word_2 = corpora.Dictionary(dataset_2)
texts_2 = dataset_2
corpus_2 = [id2word_2.doc2bow(text) for text in texts_2]

s2_model = coherence_zilla(corpus_2,id2word_2,dataset_2,texts_2,10,17)
lda_model_s2 = gensim.models.LdaMulticore(corpus=corpus_2,
                                       id2word=id2word_2,
                                       num_topics=s2_model.get('topics'), 
                                       alpha=s2_model.get('alpha'), 
                                       eta=s2_model.get('beta'), 
                                       random_state=100,
                                       chunksize=50,
                                       passes=10,
                                       per_word_topics=True)
    



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jmerc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\ProgramData\Anaconda3\envs\simple\lib\site-packages\msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)
C:\ProgramData\Anaconda3\envs\simple\lib\site-packages\msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)
C:\ProgramData\Anaconda3\envs\simple\lib\site-packages\msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)
C:\ProgramData\Anaconda3\envs\simple\lib\site-packages\msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)
C:\ProgramData\Anaconda3\envs\simple\lib\site-packages\msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated,

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.071600  0.121494       1        1  46.484451
8      0.358327 -0.044965       2        1  30.085617
3     -0.032984 -0.141876       3        1  10.207995
10    -0.109159 -0.318993       4        1   5.322764
11     0.115945  0.011723       5        1   3.569271
6     -0.056586  0.083338       6        1   1.525857
2     -0.024368  0.040339       7        1   1.163061
7     -0.058974  0.097539       8        1   0.619437
9     -0.030100  0.038956       9        1   0.255386
4     -0.033933  0.039643      10        1   0.255386
1     -0.036233  0.037713      11        1   0.255386
0     -0.020335  0.035088      12        1   0.255386, topic_info=          Term       Freq      Total Category  logprob  loglift
18    ministro  19.000000  19.000000  Default  30.0000  30.0000
44    pandemia  20.000000  20.000000  Default  29.0000  29.0000
297     c

In [9]:
pyLDAvis.enable_notebook()
LDAvis_prepared_2 = pyLDAvis.gensim.prepare(lda_model_s2, corpus_2,id2word_2)
LDAvis_prepared_2


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.071600  0.121494       1        1  46.484474
8      0.358327 -0.044965       2        1  30.085611
3     -0.032984 -0.141876       3        1  10.208006
10    -0.109159 -0.318993       4        1   5.322767
11     0.115945  0.011723       5        1   3.569204
6     -0.056586  0.083338       6        1   1.525894
2     -0.024368  0.040339       7        1   1.163066
7     -0.058974  0.097539       8        1   0.619436
9     -0.030100  0.038956       9        1   0.255386
4     -0.033933  0.039643      10        1   0.255386
1     -0.036233  0.037713      11        1   0.255386
0     -0.020335  0.035088      12        1   0.255386, topic_info=          Term       Freq      Total Category  logprob  loglift
18    ministro  19.000000  19.000000  Default  30.0000  30.0000
44    pandemia  20.000000  20.000000  Default  29.0000  29.0000
297     crisis  47.000000  47.000000  Default  28.0000  28.0000
10    gobierno  35.000000  35.000000  Default  27.0000  27.0000
28       salud  33.000000  33.000000  Default  26.0000  26.0000
..         ...        ...        ...      ...      ...      ...
44    pandemia   0.009101  20.901993  Topic12  -6.5855  -1.7691
361      gente   0.008483   8.957598  Topic12  -6.6558  -0.9920
114      datos   0.008411   7.587902  Topic12  -6.6644  -0.8347
132   expertos   0.008363   6.967904  Topic12  -6.6702  -0.7552
444  noviembre   0.008444  10.670979  Topic12  -6.6605  -1.1717

[720 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
213       1  0.334912  acciones
213       2  0.669825  acciones
72        3  0.966382     acero
57        1  0.220253   acuerdo
57        2  0.660758   acuerdo
...     ...       ...       ...
619       5  1.056151     vivio
754       2  0.995532  votacion
56        1  0.550935    vuelos
56        7  0.550935    vuelos
620       5  1.050372    vuelta

[509 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 9, 4, 11, 12, 7, 3, 8, 10, 5, 2, 1])

In [15]:
# 10 topicos -semana 2
pprint(lda_model_s2.print_topics())
print(s2_model)

[(0,
  '0.003*"medidas" + 0.003*"crisis" + 0.002*"salud" + 0.002*"trabajo" + '
  '0.002*"gobierno" + 0.002*"contagio" + 0.002*"pais" + 0.002*"proceso" + '
  '0.002*"persona" + 0.001*"autoridades"'),
 (1,
  '0.002*"crisis" + 0.002*"medidas" + 0.002*"salud" + 0.002*"pandemia" + '
  '0.002*"gobierno" + 0.002*"universidad" + 0.001*"persona" + '
  '0.001*"enfermedad" + 0.001*"noviembre" + 0.001*"pais"'),
 (2,
  '0.050*"futbol" + 0.050*"contagiarme" + 0.026*"contagio" + 0.026*"falta" + '
  '0.026*"club" + 0.026*"com" + 0.025*"conversacion" + 0.025*"pais" + '
  '0.025*"pandemia" + 0.025*"pesos"'),
 (3,
  '0.032*"jovenes" + 0.025*"contagio" + 0.025*"sintomas" + 0.018*"medidas" + '
  '0.018*"salud" + 0.018*"foco" + 0.018*"licencia" + 0.014*"persona" + '
  '0.014*"datos" + 0.014*"pais"'),
 (4,
  '0.002*"gobierno" + 0.002*"medidas" + 0.002*"salud" + 0.002*"crisis" + '
  '0.002*"contagio" + 0.002*"pandemia" + 0.001*"persona" + 0.001*"trabajo" + '
  '0.001*"ministro" + 0.001*"sintomas"'),
 (5,
  '0

In [ ]:
############# Visualización - Semana 3
s_lemma_3 = pre_proceso(df_semana_3)
dataset_3 = [d.split() for d in s_lemma_3]
id2word_3 = corpora.Dictionary(dataset_3)
texts_3 = dataset_3
corpus_3 = [id2word_3.doc2bow(text) for text in texts_3]

s3_model = coherence_zilla(corpus_3,id2word_3,dataset_3,texts_3,10,14)
lda_model_s3 = gensim.models.LdaMulticore(corpus=corpus_3,
                                       id2word=id2word_3,
                                       num_topics=s3_model.get('topics'), 
                                       alpha=s3_model.get('alpha'), 
                                       eta=s3_model.get('beta'), 
                                       random_state=100,
                                       chunksize=50,
                                       passes=10,
                                       per_word_topics=True)
    

LDAvis_prepared_3 = pyLDAvis.gensim.prepare(lda_model_s3, corpus_3,id2word_3)
LDAvis_prepared_3


In [14]:
############# Visualización - Semana 4
s_lemma_4 = pre_proceso(df_semana_4)
dataset_4 = [d.split() for d in s_lemma_4]
id2word_4 = corpora.Dictionary(dataset_4)
texts_4 = dataset_4
corpus_4 = [id2word_4.doc2bow(text) for text in texts_4]

s4_model = coherence_zilla(corpus_4,id2word_4,dataset_4,texts_4,10,14)
lda_model_s4 = gensim.models.LdaMulticore(corpus=corpus_4,
                                       id2word=id2word_4,
                                       num_topics=s4_model.get('topics'), 
                                       alpha=s4_model.get('alpha'), 
                                       eta=s4_model.get('beta'), 
                                       random_state=100,
                                       chunksize=50,
                                       passes=10,
                                       per_word_topics=True)
    



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jmerc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\ProgramData\Anaconda3\envs\simple\lib\site-packages\msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)
C:\ProgramData\Anaconda3\envs\simple\lib\site-packages\msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)
C:\ProgramData\Anaconda3\envs\simple\lib\site-packages\msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)
C:\ProgramData\Anaconda3\envs\simple\lib\site-packages\msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)
C:\ProgramData\Anaconda3\envs\simple\lib\site-packages\msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated,

In [ ]:
LDAvis_prepared_4 = pyLDAvis.gensim.prepare(lda_model_s4, corpus_4,id2word_4)
LDAvis_prepared_4
